In [20]:
from lyricsgenius import Genius
import re
from tokens import *

#artists = ['Kim Na Young', 'Heize', 'BOL4', 'OOHYO', 'Damons Year', '유라', '비비']
#cnt = 0
artists = ['moonmoon']
cnt = 384

genius = Genius(token)
for a in artists:
    res = genius.search_artist(a)

    for i in range(len(res.__dict__['songs'])):
        lyrics = res.__dict__['songs'][i].__dict__['lyrics']
        title = res.__dict__['songs'][i].title + ' - ' + a
        korean_lyrics = title + '\n' + lyrics
        with open(f'lyrics/{cnt}.txt', 'w') as f:
                f.write(korean_lyrics)
                cnt += 1
        

Searching for songs by moonmoon...

Changing artist name to 'MoonMoon'
Song 1: "비행운 (Contrail)"
Song 2: "물감 (PAINT)"
Song 3: "결혼 (Marriage)"
Song 4: "파랑새 (L’oiseau bleu)"
Song 5: "ROACH"
Song 6: "인디 (Indie)"
Song 7: "애월 (Aewol)"
Song 8: "열기구 (Balloon)"
Song 9: "모네 (Monet)"
Song 10: "사람없인 사람으로 못살아요 (People)"
Song 11: "앙고라 (Angora)"
Song 12: "나무 (Tree)"
Song 13: "우아한 세계 (Seoul)"
Song 14: "호이안 (Hoi An)"
Song 15: "휴일 (Holiday)"
Song 16: "미술관 (Museum)"
Song 17: "에덴 (Eden)"
Song 18: "디왈리 (Diwali)"
Song 19: "물고기 (Fish)"
Song 20: "맘 (Mom)"
Song 21: "순복씨 (Grand Mother)"
Song 22: "저녁향 (Evening)"
Song 23: "곰곰 (Gom Gom)"
Song 24: "아침 (Morning)"
Song 25: "FULL"
Song 26: "Moon, Moon"
Song 27: "영하 (Zero)"
Song 28: "아카시아 (Acacia)"
Song 29: "푸욱 [Poo Ooq] (of moon)"
Song 30: "푸욱 [Poo Ooq] (of mars)"
"미술관 (Museum) [Instrumental]" is not valid. Skipping.
"결혼 (Marriage) [Instrumental]" is not valid. Skipping.
Done. Found 30 songs.


In [1]:
def convert_korean_punctuation(text):
    # Mapping of Korean punctuation to English equivalents
    conversion_mapping = {
        '\u2018': "'",  # Left single quotation mark
        '\u2019': "'",  # Right single quotation mark
        '\u201C': '"',  # Left double quotation mark
        '\u201D': '"',  # Right double quotation mark
        '\u2026': '...', # Horizontal ellipsis
        '\u3001': ',',  # Ideographic comma
        '\u3002': '.',  # Ideographic full stop
        '\u00A0': ' ',  # No-Break Space
        '\u1680': ' ',  # Ogham Space Mark
        '\u180E': ' ',  # Mongolian Vowel Separator
        '\u2000': ' ',  # En Quad
        '\u2001': ' ',  # Em Quad
        '\u2002': ' ',  # En Space
        '\u2003': ' ',  # Em Space
        '\u2004': ' ',  # Three-Per-Em Space
        '\u2005': ' ',  # Four-Per-Em Space
        '\u2006': ' ',  # Six-Per-Em Space
        '\u2007': ' ',  # Figure Space
        '\u2008': ' ',  # Punctuation Space
        '\u2009': ' ',  # Thin Space
        '\u200A': ' ',  # Hair Space
        '\u200B': ' ',  # Zero Width Space
        '\u202F': ' ',  # Narrow No-Break Space
        '\u205F': ' ',  # Medium Mathematical Space
        '\u3000': ' ',  # Ideographic Space
        '\uFEFF': ' ',  # Zero Width No-Break Space
    }

    return text

In [2]:
import json
# read the dictionary title_ids from title_ids.txt
title_ids = {}
with open('title_ids.json', 'r') as f:
    title_ids = json.load(f)

In [6]:
import re
import os
import regex

folder_path = "lyrics/"
lyrics_dir = "lyrics/"
i = 0
missing_ids = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and file_path.endswith(".txt"):
        # open the file and read it
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            text = convert_korean_punctuation(text)
        lines = text.split("\n")
        lyrics = '\n'.join(lines[1:])
        title = convert_korean_punctuation(lines[0])
        # remove english from the last line
        lyrics = lyrics.split('\n')
        lyrics[0] = regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', lyrics[0])[-1]
        lyrics[-1] = regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', lyrics[-1])[0]
        korean_lyrics = '\n'.join(lyrics)
        korean_lyrics = korean_lyrics.strip().replace('\n\n', '\n')
        
        newlines = []
        for line in lyrics:
            if re.search(r'[a-zA-Z\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]', line):
                if not re.search(r'[a-zA-Z\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]', regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', line)[-1]):
                    newlines.append(regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', line)[-1])
                elif not re.search(r'[a-zA-Z\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]', regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', line)[0]):
                    newlines.append(regex.split(r'[a-zA-Z]+|[ぁ-んァ-ン一-龥]|\p{P}', line)[0])
                else: newlines.append('')
            else:
                newlines.append(line)
                
        korean_lyrics = '\n'.join(line.strip() for line in newlines if not re.search(r'[a-zA-Z\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]', line))
        korean_lyrics = '\n'.join(line.strip() for line in korean_lyrics.split('\n') if len(line.split()) > 1)
        if len(korean_lyrics.split('\n')) > len(lines) * .6: 
            if title in title_ids:
                id = title_ids[title]
            else: id = None
            if id == None:
                missing_ids.append(title)
            json_data = {
                "title": title,
                "lyrics": korean_lyrics.split('\n'),
                "video_id": id
            }

            # Write the JSON object to a JSON file
            json_file_path = os.path.join(lyrics_dir, f"{i}.json")
            with open(json_file_path, 'w', encoding='utf-8') as json_file:
                json.dump(json_data, json_file, ensure_ascii=False, indent=2)
            
            i += 1
print("Missing ids: " + str(missing_ids))
print("Total videos: " + str(i))


Missing ids: []
Total videos: 222


In [5]:
from googleapiclient.discovery import build
import json
import os
from tokens import api_key1, api_key2, api_key3

youtube = build("youtube", "v3", developerKey=api_key1)

# loop through missing ids
for id in missing_ids:
    # Assuming the song title is the first line in the "lyrics" field
    song_title = id

    # Search for videos for the song title
    search_response = youtube.search().list(
        q=song_title,
        type="video",
        part="id",
        maxResults=1
    ).execute()

    if len(search_response['items']) == 0:
        print(f"No videos found for '{song_title}'")
        continue
    else:
        # add the video id to the json file
        title_ids[song_title] = search_response['items'][0]['id']['videoId']

In [7]:
with open('title_ids.json', 'w') as f:
    json.dump(title_ids, f, indent=4)

In [21]:
import json
# loop through lyrics/n.json for n in range(0, 168) and put their video ids in a list
title_ids = {}
for i in range(0, 173):
    with open('lyrics/' + str(i) + '.json', 'r') as f:
        data = json.load(f)
        title = data['title']
        id = data['video_id']
        title_ids[title] = id
#write video_ids to a text file
with open('title_ids.json', 'w') as f:
    json.dump(title_ids, f, indent=4)

In [ ]:
from googleapiclient.discovery import build
import json
import os
from tokens import api_key1, api_key2, api_key3

youtube = build("youtube", "v3", developerKey=api_key1)

folder_path = "lyrics" # Path to the folder containing the JSON files

n_songs = 168
cnt = 168
# Iterate through the files in the specified range
for n in range(0, n_songs):
    file_path = os.path.join(folder_path, f"{n}.json")

    # Check if the file exists
    if os.path.isfile(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # Assuming the song title is the first line in the "lyrics" field
            song_title = data["title"]
            
            # Search for videos for the song title
            search_response = youtube.search().list(
                q=song_title,
                type="video",
                part="id",
                maxResults=1
            ).execute()

            if len(search_response['items']) == 0:
                print(f"No videos found for '{song_title}'")
                continue
            else:
                # add the video id to the json file
                data["video_id"] = search_response['items'][0]['id']['videoId']
                with open(f"lyrics/{cnt}.json", 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)
                    cnt += 1


print("Finished converting videos. Missing ids: " + missing_ids)


KeyboardInterrupt: 